In [1]:
import pandas as pd 
import numpy as np
import re
import os
import shutil
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("expand.frame_repr",False)
print('import succed!')

import succed!


In [2]:
input_path = r'/Users/edz/Documents/yandi_data/AI_data'
input_path_file = r'/patient3.csv'

In [3]:
df_ym= pd.read_csv(input_path + input_path_file,sep=',',index_col=None,header=None)
print(df_ym.shape)   # 现有库存数据中“影像描述”不为空的数据容量

(250625, 16)


/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# 修改列名
df_ym.columns =  ['来自检查id','送阅时间','病例医院名称','阅片医院名称','病历医生id','病例医生名称','阅片医生id','阅片医生名称',
                '病史','全身病','病例诊断','影像描述','阅片诊断','质控影像描述','质控阅片诊断','主诉、病例检查、处理、处方json串']
df_ym.tail(2) #校验列名修改成功

,来自检查id,送阅时间,病例医院名称,阅片医院名称,病历医生id,病例医生名称,阅片医生id,阅片医生名称,病史,全身病,病例诊断,影像描述,阅片诊断,质控影像描述,质控阅片诊断,主诉、病例检查、处理、处方json串
250623,11018815,2019-03-04 14:54:11,河北省保定市东风社区卫生服务中心,至真阅片中心,4752,白月红,82,白海霞,NaN,NaN,NaN,双眼视盘边清色正，盘沿形态未见明确异常；视网膜血管走行正常；黄斑区未见明显异常。,双眼眼底未见明显异常,NaN,NaN,"{""complain"":"""",""impression"":"""",""deal"":"""",""pres..."
250624,11018943,2019-03-07 09:09:27,昆明市西山区永昌社区卫生服务中心,至真阅片中心,65,李玲玲,12,崔馨,NaN,NaN,NaN,双眼视盘边清色正，盘沿形态未见明确异常；左眼及右眼可见部分视网膜血管走行正常，未见异常新生血...,右眼眼底未见明显异常；左眼眼底窥不清,NaN,NaN,"{""complain"":"""",""impression"":"""",""deal"":"""",""pres..."


In [5]:
df_ym.duplicated(subset='来自检查id').sum()  #判断指定Series的行数据是否存在重复项
# 或者 df_qgy['来自检查id'].duplicated().sum() 

0

In [6]:
df_ym['影像描述'] = df_ym['影像描述'].fillna('nan') # 空缺项填充nan
df_ym['影像描述'].isnull().sum()

0

In [7]:
# df_ym[df_ym['影像描述'].str.contains('双眼视盘边清色正',re.I)]['影像描述'].value_counts()  

In [8]:
# 自定义”全角转半角“函数
def strQ2B(ustring):
    """把字符串全角转半角"""
    ss = []
    for s in ustring:   # ustring=['后发障','AION','无法诊断'],s=list中依次取得的各个字符串
        rstring = ""
#         print("每一行值s:{}".format(s)) 
        for uchar in s:  # s='后发障',uchar='后发障'中依次取得的各个字符
            inside_code = ord(uchar)
#                 print(inside_code)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
#             print(rstring)
        ss.append(rstring)
#         print(ss)
    return ss
#     return ''.join(ss)

In [9]:
#调用函数，实现全角转半角
df_ym['影像描述'] = strQ2B(df_ym['影像描述'])  # ''.join用于合并字符，df_ym.at['影像描述'] = ''.join(strQ2B(str_i))
df_ym.loc[:5,'影像描述']

0    视盘边清色淡,右眼C/D=0.7,颞侧盘沿消失,双眼黄斑中心光未见,视网膜动脉细,反光强,交...
1    双眼视盘边清色正,盘沿形态未见明确异常;视网膜血管走行正常,未见异常新生血管;右眼视网膜散在...
2    双眼视盘边清色正,盘沿形态未见明确异常;视网膜动脉管径普遍变细,A/V=1:3,视网膜动脉管...
3                                  双眼后极部散在点片状出血灶,黄白色病灶
4                          可见双眼视盘边清色淡,黄斑中心光未见,视网膜呈豹纹状。
5               双眼视盘边界清,色正常,视网膜血管走行正常,未见出血及渗出,中心凹反射不清。
Name: 影像描述, dtype: object

In [10]:
# 插入'影描_gai'列
df_ym.insert(df_ym.columns.get_loc('影像描述')+1,'影描_gai',df_ym['影像描述'])

## 去除空格、括号、波浪线等
\s表示空格，b = re.compile('\s')表示匹配空格；df_tw.at[i,'影描_gai'] = re.compile('\s').sub('',str_i)
匹配到空格之后替换为“”（即：去除空格）

In [11]:
# 去除字符串中的“\n|\t|\r|~|——”
def kong_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
    #     match_str = '(左|右\d期)' 
        if str_i == 'nan':
            pass
        elif len(re.compile('\n|\t|\r|~|——|(|)|,|、|。|\?').findall(str_i)) > 0: # 去除字符串中的“\n|\t|\r|~|——”
#             print(str_i)
            str_i_1 = str_i.replace('\n','').replace('\r','').replace('\t','').replace(',','').replace('。','').replace('、','')    # 去除\n|\t|\r和,以及。   
            str_i_2 = str_i_1.replace('~','-').replace('——','-')  # 改~和————为短中线
            str_i_3 = re.compile('\?').sub('',str_i_2)
            str_i_4 = re.compile(';').sub('',str_i_3)
            df.at[i,'影描_gai'] = str_i_4.replace('(','').replace(')','')  # 去除括号
#             print('__{}'.format(df.at[i,'影描_gai']))
        else:
            pass

    check_df = df[df['影描_gai'].str.contains('\n|\t|\r|~|——|\(|\)|,|、|。|\?|;')]
    if check_df.shape[0] > 0:
        print("剩余{}条数据存在无效标点".format(check_df.shape[0]))
        kong_fuc(df)
    else:
        print('转化结束！！！')
        pass

In [12]:
kong_fuc(df_ym)    # 调用去除无效标点的函数

转化结束！！！


In [13]:
# 去除空格
def kong1_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        elif len(re.compile('\s').findall(str_i)) > 0:
    #         print(str_i)
            df.at[i,'影描_gai'] = re.compile('\s').sub('',str_i)  # 去除字符串首尾的空格
    #         print('__{}'.format(df.at[i,'影描_gai']))
        else:
            pass

    check_df = df[df['影描_gai'].str.contains('\s')]  #校验
    if check_df.shape[0] > 0:
        print("剩余{}条数据存在空格".format(check_df.shape[0]))
        kong1_fuc(df)
    else:
        print('转化结束！！！')
        pass

In [14]:
kong1_fuc(df_ym)

转化结束！！！


In [15]:
########### 校验是否还存在空格
for i in range(df_ym.shape[0]):
    str_i = str(df_ym.at[i,'影描_gai'])
    if str_i == 'nan':
        pass
    elif len(re.compile('\n|\t|\r|~|——|\(|\)|,|、|。|\?|;').findall(str_i)) > 0:
        print(str_i)
        df_ym.at[i,'影描_gai'] = re.compile('\?').sub('',str_i)  # 去除字符串首尾的空格
        print('__{}'.format(df_ym.at[i,'影描_gai']))
    else:
        pass

In [16]:
# 增加“分隔符”的自定义函数
def add_function(x):
    import re
    list_x=[i.start() for i in re.finditer('双|左|右', x)]
#     print(list_x)

    for i in range(len(list_x)):
        x = list(x)
    #     print(list_x[i])
        x.insert(list_x[i]+i,'!')
        x ="".join(x)
#     print(x)
    return x

In [17]:
#调用函数，增加分隔符
for m in range(df_ym.shape[0]):
    df_ym.at[m,'影描_gai'] = add_function(df_ym.at[m,'影描_gai'])

print(df_ym.loc[:5,'影描_gai'])  #校验

0    视盘边清色淡!右眼C/D=0.7颞侧盘沿消失!双眼黄斑中心光未见视网膜动脉细反光强交叉征+余...
1    !双眼视盘边清色正盘沿形态未见明确异常视网膜血管走行正常未见异常新生血管!右眼视网膜散在玻璃...
2    !双眼视盘边清色正盘沿形态未见明确异常视网膜动脉管径普遍变细A/V=1:3视网膜动脉管壁反光...
3                                  !双眼后极部散在点片状出血灶黄白色病灶
4                            可见!双眼视盘边清色淡黄斑中心光未见视网膜呈豹纹状
5                   !双眼视盘边界清色正常视网膜血管走行正常未见出血及渗出中心凹反射不清
Name: 影描_gai, dtype: object


In [18]:
## 缺少左右眼的——添加“双”标签
# 处理字符串没有“左/右/双”的情形
def lr_shuang_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        else:
            str_tempt = re.split('!',str_i)
            str_i_list = list(filter(None,str_tempt))
            str_new_list = []
            for j in range(len(str_i_list)):
                if len(re.compile('(右|左|双)').findall(str_i_list[j])) == 0:
#                     print(str_i_list)
                    str_new_list.append('双' + str_i_list[j])

                else:
                    str_new_list.append(str_i_list[j])
            df.at[i,'影描_gai'] = '!'.join(str_new_list[m] for m in range(len(str_new_list)))
#             print("___{}".format(df.at[i,'影描_gai']))

In [19]:
lr_shuang_fuc(df_ym)  ## 调用添加“双”标签的函数

In [20]:
############## 验证“双”补全
for i in range(df_ym.shape[0]):
    str_i = str(df_ym.at[i,'影描_gai'])
    if str_i == 'nan':
        pass
    else:
        str_tempt = re.split('!',str_i)
        str_i_list = list(filter(None,str_tempt))
        str_new_list = []
        for j in range(len(str_i_list)):
            if len(re.compile('(双|左|右)').findall(str_i_list[j])) == 0:
                print(str_i_list[j])

In [21]:
print("影描_gai——的规格：{}".format(df_ym['影描_gai'].shape[0]))
print("影像描述——的规格：{}".format(df_ym['影像描述'].shape[0]))

影描_gai——的规格：250625
影像描述——的规格：250625


# 通用字符串的名称归一化

In [22]:
# 定义归一化函数——单个标签替换
def single_replace_fuc(patten_x, repl_x,string_x):
    strinfo_0 = re.compile(patten_x, re.I)
    if len(strinfo_0.findall(string_x)) > 0:
#         print(string_x)
        string_x = re.sub(re.compile(patten_x, flags=re.I), repl_x, string_x)
#         print("__{}".format(string_x))
    else:
        pass
    
    return string_x

In [23]:
####定义”双眼\|右眼\|左眼“转为”双|左|右“的转化函数
#双、左、右眼——名称统一
# 为解决调用一次函数，会出现的“修改不完全的问题”，可以多次调用函数处理遗留的未匹配数据。
def lrs_normal_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        else:
            str_i_1 = single_replace_fuc('双眼', '双', str_i)  #str_i_1/2/3均为中间变量,调用函数 single_replace_fuc
            str_i_2 = single_replace_fuc('左眼', '左', str_i_1)
            str_i_3 = single_replace_fuc('右眼', '右', str_i_2)
            df.at[i,'影描_gai'] =  str_i_3

    check_df = df[df['影描_gai'].str.contains('双眼|右眼|左眼')]
    if check_df.shape[0] > 0:
        print("剩余{}条数据包含双眼|右眼|左眼".format(check_df.shape[0]))
        lrs_normal_fuc(df)
    else:
        print('转化结束！！！')
        pass

In [24]:
lrs_normal_fuc(df_ym)    # 调用“”双眼\|右眼\|左眼“转为”双|左|右“的转化函数

剩余49416条数据包含双眼|右眼|左眼
剩余8条数据包含双眼|右眼|左眼
转化结束！！！


In [25]:
df_ym[df_ym['影描_gai'].str.contains('网膜隐见出血')]['影描_gai']

9479      双晶体混浊眼底模糊!右著!双视盘色淡红界清C/D!右0.5!左0.4!右颞下网膜隐见出血!左...
31550                          双底模糊!左可见处隐约未见明显异常!右颞上方网膜隐见出血
41757                                    双晶体混浊眼底像模糊网膜隐见出血渗出
41886                     双图像视野不全!双视网膜动静脉可见交叉压迹!双视网膜隐见出血及硬渗
44940       双晶体皮质不均匀混浊眼底模糊视乳头色界正常!右黄斑前出血!左黄斑未见明显异常视网膜隐见出血斑点
45812                      右底像模糊隐见视盘网膜隐见出血斑余窥不清!左视网膜出血渗出性改变
47104     双底像模糊暗!右视盘黄斑窥不清颞上周边网膜可见陈旧激光斑出血渗出!左隐见视盘边清色正盘沿形态...
48536     右外斜虹膜组织模糊瞳孔3mm眼底不入!左晶体皮质不均匀混浊眼底模糊视乳头色界正常杯盘比0.3...
49881         双视盘边清色可视网膜动静脉走行及比例大致正常!右网膜见出血渗出激光斑!左网膜隐见出血激光斑
68329                    双底模糊!右底像位置不正视盘显示不全!左视盘界清色可!双网膜隐见出血
73257                       双底像欠清晰视盘边清色可视网膜动脉缩窄黄斑区看不清网膜隐见出血
81938                  双图像视野不全!双屈光间质不清!右视网膜隐见出血!左视网膜可见出血及硬渗
87816               双晶体混浊!左玻璃体似有出血混浊视网膜隐见出血!右视网膜动脉缩窄硬化视网膜前膜
88731                  双底模糊!右隐见视盘和大血管网膜细节看不清!左视盘界清色可网膜隐见出血斑
89339                          双视乳头色界正常黄斑未见明显异常!左鼻侧视网膜隐见出血斑
116445                    双底模糊视乳头色界正常黄斑可!双视网膜隐见出血及黄白色斑点!右明显
120828                          双底像欠清晰视盘

In [30]:
# 查看不同类型的名称
value_list = []
for i in range(df_ym.shape[0]):
    str_i = str(df_ym.at[i,'影描_gai'])
    if str_i=='nan':   # 排除str_i为”null“的项
        pass
    else:
        str_tempt = re.split(r'!', str_i)
        str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
        for j in range(len(str_i_list)):
            if len(re.compile('视网膜前.出血').findall(str_i_list[j]))>0 and len(re.compile('未见出血|未见..出血|未见渗出及出血').findall(str_i_list[j]))==0:
#                 value_list.append(str_i_list[j])
                if len(re.compile('_视网膜前出血').findall(str_i_list[j]))==0 and len(re.compile('视网膜前未见').findall(str_i_list[j]))==0:
                    value_list.append(str_i_list[j])
#                 print(str_i_list[j])
                else:
                    pass
            else:
                pass
value_list = set(value_list)
# print(value_list)    

In [31]:
list(value_list) # 

['右视网膜前膜出血', '右视盘前视网膜前膜出血', '右黄斑区视网膜前下出血上方动脉白色病灶']

#### MA表示微血管瘤，NP是无灌注区，NV表示新生血管。

##### 散在：一般情况下是点状为主；片状一般是一大块呈斑状；火焰状更多的是形容颜色和纹理，带有形状和状态

神经纤维层的出血又称——火焰状出血（Flame-shaped Hemorrhages）很常见并且易于观察，出血沿着神经纤维呈线性分布，形似裂片（splinters），因此又叫裂片样出血（splinter hemorrhages）

视网膜出血是许多眼底疾病的显著体征之一.
#### 按部位分类：
有周边部、赤道部、后极部、视盘周围视网膜出血和玻璃体积血等;
#### 按形态分类：
有点状、片状、火焰状视网膜出血等;
#### 按层次分类：
有视网膜前出血、内界膜下出血、浅层出血(神经纤维层出血)、深层出血、视网膜下出血(神经上皮下出血)和视网膜色素上皮(RPE)下出血等

### 出血_location——名称归一化

In [ ]:
####定义_视网膜出血——名称统一
def normal_cd_fuc(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        else:
            str_tempt = re.split(r'!', str_i)
            str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
            value_list = []
            for j in range(len(str_i_list)):
#                 print(str_i_list)
                if len(re.compile('出血').findall(str_i_list[j]))>0 and len(re.compile('未见明显MA及上述病灶外的出血|未见明显异常..出血|未见明显MA及出血|未见明确MA及出血|未见明显MA出血|未见明显渗出及出血|无明显出血渗出|未见出血|未见..出血|未见渗出及出血|未见大出血|未见MA及出血点').findall(str_i_list[j]))==0:
                    if len(re.compile('视网膜下|视网膜内出血|视网膜散出血|视网膜有出血|视网膜间出血|视网膜前出血|视网膜下出血|视网膜见出血|视网膜平出血|视网膜小出血|视网膜除出血').findall(str_i_list[j]))>0:
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜下|视网膜内出血|视网膜散出血|视网膜有出血|视网膜间出血|视网膜前出血|视网膜下出血|视网膜见出血|视网膜平出血|视网膜小出血|视网膜除出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 均是汉字的匹配不能加re.I,否则识别不出来
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜出血').sub('出血_视网膜出血',str_i_list[j])               
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜大量出血|视网膜片状出血|视网膜线状出血|视网膜点状出血|视网膜可见出血|视网膜散在出血|视网膜较多出血|视网膜平见出血|视网膜广泛出血|视网膜大片出血|视网膜小片出血|视网膜少量出血|视网膜隐见出血|视网膜下方出血|视网膜似有出血|视网膜偶见出血|视网膜可疑出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 均是汉字的匹配不能加re.I,否则识别不出来
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜大量出血|视网膜片状出血|视网膜线状出血|视网膜点状出血|视网膜可见出血|视网膜散在出血|视网膜较多出血|视网膜平见出血|视网膜广泛出血|视网膜大片出血|视网膜小片出血|视网膜少量出血|视网膜隐见出血|视网膜下方出血|视网膜似有出血|视网膜偶见出血|视网膜可疑出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜少许出血|视网膜水肿出血|视网膜线样出血|视网膜簇状出血|视网膜前下出血|视网膜疑似出血|视网膜数个出血|视网膜满布出血|视网膜.个出血点|视网膜颞上出血|视网膜颞下出血|视网膜深层出血|视网膜浅层出血|视网膜圆形出血|视网膜多处出血|视网膜陈旧出血|视网膜多片出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 均是汉字的匹配不能加re.I,否则识别不出来
                        str_i_list[j] = re.compile('视网膜少许出血|视网膜水肿出血|视网膜线样出血|视网膜簇状出血|视网膜前下出血|视网膜疑似出血|视网膜数个出血|视网膜满布出血|视网膜.个出血点|视网膜颞上出血|视网膜颞下出血|视网膜深层出血|视网膜浅层出血|视网膜圆形出血|视网膜多处出血|视网膜陈旧出血|视网膜多片出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜见一出血|视网膜数片出血|视网膜多处出血|视网膜硬渗出血|视网膜浓密出血|视网膜条状出血|视网膜点片出血|视网膜局部出血|视网膜小的出血|视网膜上方出血|视网膜斑点出血|视网膜眼底出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 均是汉字的匹配不能加re.I,否则识别不出来
                        str_i_list[j] = re.compile('视网膜见一出血|视网膜数片出血|视网膜多处出血|视网膜硬渗出血|视网膜浓密出血|视网膜条状出血|视网膜点片出血|视网膜局部出血|视网膜小的出血|视网膜上方出血|视网膜斑点出血|视网膜眼底出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜掐脑出血|视网膜大批出血|视网膜轻度出血|视网膜条形出血|视网膜.个出血|视网膜见小出血|视网膜片装出血|视网膜期脑出血|视网膜密集出血|视网膜可加出血|视网膜焰状出血|视网膜一小出血|视网膜后极出血|视网膜多量出血|视网膜颞侧出血|视网膜表面出血|视网膜浓厚出血|视网膜小状出血|视网膜渗出出血|视网膜团状出血|视网膜现状出血|视网膜多发出血|视网膜可以出血|视网膜MA出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜掐脑出血|视网膜大批出血|视网膜轻度出血|视网膜条形出血|视网膜.个出血|视网膜见小出血|视网膜片装出血|视网膜期脑出血|视网膜密集出血|视网膜可加出血|视网膜焰状出血|视网膜一小出血|视网膜后极出血|视网膜多量出血|视网膜颞侧出血|视网膜表面出血|视网膜浓厚出血|视网膜小状出血|视网膜渗出出血|视网膜团状出血|视网膜现状出血|视网膜多发出血|视网膜可以出血|视网膜MA出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜数个小出血|视网膜平见小出血|视网膜颞.方出血|视网膜前大片出血|视网膜可见小出血点|视网膜.片状出血|视网膜可见小出血|视网膜火焰状出血|视网膜下圆形出血|视网膜可疑小出血|视网膜点片状出血|视网膜下浓密出血|视网膜下大.出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜数个小出血|视网膜平见小出血|视网膜颞.方出血|视网膜前大片出血|视网膜可见小出血点|视网膜.片状出血|视网膜可见小出血|视网膜火焰状出血|视网膜下圆形出血|视网膜可疑小出血|视网膜点片状出血|视网膜下浓密出血|视网膜下大.出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜平点状出血|视网膜小点状出血|视网膜下增殖出血|视网膜疑似一出血点|视网膜平鼻下出血|可视网膜散在.出血|视网膜大.前出血|视网膜前大.出血|视网膜数小片出血|视网膜下可见出血|视网膜各项限出血|视网膜大片装出血|视网膜一小片出血|视网膜前大.出血|视网膜可见片出血|视网膜平散在出血|视网膜可大量出血|视网膜.小片出血|视网膜MA及出血|视网膜放射状出血|视网膜硬渗及出血|视网膜一点状出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜平点状出血|视网膜小点状出血|视网膜下增殖出血|视网膜疑似一出血点|视网膜平鼻下出血|可视网膜散在.出血|视网膜大.前出血|视网膜前大.出血|视网膜数小片出血|视网膜下可见出血|视网膜各项限出血|视网膜大片装出血|视网膜一小片出血|视网膜前大.出血|视网膜可见片出血|视网膜平散在出血|视网膜可大量出血|视网膜.小片出血|视网膜MA及出血|视网膜放射状出血|视网膜硬渗及出血|视网膜一点状出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜平多处出血|视网膜见点状出血|视网膜深浅层出血|视网膜前散在出血|视网膜大面积出血|视网膜陈旧性出血|视网膜圆点状出血|视网膜下片装出血|视网膜平细小出血点|视网膜斑点状出血|视网膜疑似小出血|视网膜下环状出血|视网膜被大片出血|视网膜散在小出血|视网膜裂隙状出血|视网膜平网膜出血|视网膜下多量出血|视网膜隐见小出血|视网膜弥漫性出血|视网膜可见一出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜平多处出血|视网膜见点状出血|视网膜深浅层出血|视网膜前散在出血|视网膜大面积出血|视网膜陈旧性出血|视网膜圆点状出血|视网膜下片装出血|视网膜平细小出血点|视网膜斑点状出血|视网膜疑似小出血|视网膜下环状出血|视网膜被大片出血|视网膜散在小出血|视网膜裂隙状出血|视网膜平网膜出血|视网膜下多量出血|视网膜隐见小出血|视网膜弥漫性出血|视网膜可见一出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜似水肿出血|视网膜隐见一出血|视网膜可见状出血|视网膜像大量出血|视网膜大MA出血|视网膜区域性出血|视网膜.状小出血|视网膜及深层出血|视网膜较多量出血|视网膜下渗出出血|视网膜斑点样出血|视网膜前.量出血|视网膜前可见出血|视网膜可见点片状出血|视网膜钱大量出血|视网膜大范围出血|视网膜棉絮斑出血|视网膜渗出及出血|视网膜颞广泛出血|视网膜前似有出血|视网膜被大量出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 
                        str_i_list[j] = re.compile('视网膜似水肿出血|视网膜隐见一出血|视网膜可见状出血|视网膜像大量出血|视网膜大MA出血|视网膜区域性出血|视网膜.状小出血|视网膜及深层出血|视网膜较多量出血|视网膜下渗出出血|视网膜斑点样出血|视网膜前.量出血|视网膜前可见出血|视网膜可见点片状出血|视网膜钱大量出血|视网膜大范围出血|视网膜棉絮斑出血|视网膜渗出及出血|视网膜颞广泛出血|视网膜前似有出血|视网膜被大量出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜下陈旧出血|视网膜下液及出血|视网膜一局限出血|视网膜可疑有出血|视网膜混浊及出血|视网膜见多处出血|视网膜颞侧小出血|视网膜见一小出血|视网膜见MA出血|视网膜隐约见出血|视网膜反射状出血|视网膜钱大片出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 
                        str_i_list[j] = re.compile('视网膜下陈旧出血|视网膜下液及出血|视网膜一局限出血|视网膜可疑有出血|视网膜混浊及出血|视网膜见多处出血|视网膜颞侧小出血|视网膜见一小出血|视网膜见MA出血|视网膜隐约见出血|视网膜反射状出血|视网膜钱大片出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜可见.状出血|视网膜..少量出血|视网膜可见..出血|视网膜偶见.状出血|视网膜及网膜下出血|视网膜散在.量出血|视网膜散在.状出血|视网膜..MA出血|视网膜微血管瘤出血|视网膜深层.状出血|视网膜可见.片出血|视网膜可疑.片出血|视网膜可见MA出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜可见.状出血|视网膜..少量出血|视网膜可见..出血|视网膜偶见.状出血|视网膜及网膜下出血|视网膜散在.量出血|视网膜散在.状出血|视网膜..MA出血|视网膜微血管瘤出血|视网膜深层.状出血|视网膜可见.片出血|视网膜可疑.片出血|视网膜可见MA出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜颞上.片出血|视网膜可疑.形出血|视网膜.方.状出血|视网膜散在硬渗出血|视网膜散在.形出血|视网膜广泛水肿出血|视网膜见MA及出血|视网膜.量.状出血|视网膜内点片状出血|视网膜水肿散在出血|视网膜多发散在出血|视网膜可疑.状出血|视网膜.方可疑出血|视网膜散在.片出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜颞上.片出血|视网膜可疑.形出血|视网膜.方.状出血|视网膜散在硬渗出血|视网膜散在.形出血|视网膜广泛水肿出血|视网膜见MA及出血|视网膜.量.状出血|视网膜内点片状出血|视网膜水肿散在出血|视网膜多发散在出血|视网膜可疑.状出血|视网膜.方可疑出血|视网膜散在.片出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜数小.状出血|视网膜少许片状出血|视网膜隐约可见出血|视网膜隐见.状出血|视网膜颞上广泛出血|视网膜隐见多处出血|视网膜.片渗出出血|视网膜隐见.量出血|视网膜平网膜见出血|可视网膜平可疑小出血|视网膜水肿线状出血|视网膜多处散在出血|视网膜下大范围出血|视网膜广泛硬渗出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜数小.状出血|视网膜少许片状出血|视网膜隐约可见出血|视网膜隐见.状出血|视网膜颞上广泛出血|视网膜隐见多处出血|视网膜.片渗出出血|视网膜隐见.量出血|视网膜平网膜见出血|可视网膜平可疑小出血|视网膜水肿线状出血|视网膜多处散在出血|视网膜下大范围出血|视网膜广泛硬渗出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜多处.状出血|视网膜广泛大量出血|视网膜水肿大量出血|视网膜深层浅层出血|视网膜.侧可见出血|视网膜MA.片出血|视网膜小血管瘤出血|视网膜水肿小片出血|视网膜前多平面出血|视网膜后极.片出血|视网膜较多.状出血|视网膜较.片状出血|网膜疑似.状出血|视网膜散在数个出血|视网膜浅层.状出血|视网膜脱离偶见出血|视网膜似有.状出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜多处.状出血|视网膜广泛大量出血|视网膜水肿大量出血|视网膜深层浅层出血|视网膜.侧可见出血|视网膜MA.片出血|视网膜小血管瘤出血|视网膜水肿小片出血|视网膜前多平面出血|视网膜后极.片出血|视网膜较多.状出血|视网膜较.片状出血|网膜疑似.状出血|视网膜散在数个出血|视网膜浅层.状出血|视网膜脱离偶见出血|视网膜似有.状出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜下增殖膜出血|视网膜.量散在出血|视网膜.上疑似出血|视网膜可疑微小出血|视网膜下及表面出血|视网膜.方疑似出血|视网膜.方似有出血|双视网膜似可见小出血点|视网膜广泛片状出血|视网膜仍有.状出血|视网膜.下.片出血|视网膜广泛散在出血|视网膜MA及.出血|视网膜.量散在出血|视网膜多发.状出血|视网膜.侧可疑出血|视网膜.量.片出血|视网膜散在分布出血|视网膜.侧可疑出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜下增殖膜出血|视网膜.量散在出血|视网膜.上疑似出血|视网膜可疑微小出血|视网膜下及表面出血|视网膜.方疑似出血|视网膜.方似有出血|双视网膜似可见小出血点|视网膜广泛片状出血|视网膜仍有.状出血|视网膜.下.片出血|视网膜广泛散在出血|视网膜MA及.出血|视网膜.量散在出血|视网膜多发.状出血|视网膜.侧可疑出血|视网膜.量.片出血|视网膜散在分布出血|视网膜.侧可疑出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜颞侧疑似出血|视网膜似有一.出血|视网膜.大片状出血|视网膜颞上线状出血|视网膜平见散在出血|视网膜下病灶周出血|方视网膜浅层深层出血|视网膜.方散在出血|视网膜.方隐见出血|视网膜少许.状出血|视网膜上方见一出血|视网膜呈放射状出血|视网膜各见小片出血|视网膜见斑片状出血|视网膜线状点状出血|视网膜萎缩片状出血|视网膜.方.片出血|视网膜散在陈旧出血|视网膜条状片状出血|视网膜隐见较多出血').findall(str_i_list[j]))>0 and len(re.compile('出血_视网膜出血').findall(str_i_list[j]))==0:  # 
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜颞侧疑似出血|视网膜似有一.出血|视网膜.大片状出血|视网膜颞上线状出血|视网膜平见散在出血|视网膜下病灶周出血|方视网膜浅层深层出血|视网膜.方散在出血|视网膜.方隐见出血|视网膜少许.状出血|视网膜上方见一出血|视网膜呈放射状出血|视网膜各见小片出血|视网膜见斑片状出血|视网膜及视盘前出血|视网膜及玻璃体出血|视网膜线状点状出血|视网膜萎缩片状出血|视网膜.方.片出血|视网膜散在陈旧出血|视网膜条状片状出血|视网膜隐见较多出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜隐见几处出血|视网膜.状.状出血|视网膜平点片状出血|视网膜.片水肿出血|视网膜水肿渗出出血|视网膜隐见.片出血|视网膜一处片状出血|视网膜偶见.片出血|视网膜.量硬渗出血|视网膜上方可见出血|视网膜似有散在出血|视网膜皱褶网膜出血|视网膜大片致密出血|视网膜见.片状出血|视网膜后极部见出血|视网膜周边散在出血|视网膜.方浓密出血|视网膜浅斑点状出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜隐见几处出血|视网膜.状.状出血|视网膜平点片状出血|视网膜.片水肿出血|视网膜水肿渗出出血|视网膜隐见.片出血|视网膜一处片状出血|视网膜偶见.片出血|视网膜.量硬渗出血|视网膜上方可见出血|视网膜似有散在出血|视网膜皱褶网膜出血|视网膜大片致密出血|视网膜见.片状出血|视网膜后极部见出血|视网膜周边散在出血|视网膜.方浓密出血|视网膜浅斑点状出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜上下方见出血|视网膜各一小片出血|视网膜似有大量出血|视网膜.上可见出血|视网膜呈火焰状出血|视网膜似有.片出血|视网膜前增殖膜出血|视网膜大片火焰出血|视网膜.多片状出血|视网膜多处细小出血|视网膜有细点状出血|视网膜区域可见出血|视网膜渗出少许出血|视网膜棉绒斑及出血|视网膜.方意思出血|视网膜大量斑点出血|视网膜水肿.状出血|视网膜暗红色灶出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜上下方见出血|视网膜各一小片出血|视网膜似有大量出血|视网膜.上可见出血|视网膜呈火焰状出血|视网膜似有.片出血|视网膜前增殖膜出血|视网膜大片火焰出血|视网膜.多片状出血|视网膜多处细小出血|视网膜有细点状出血|视网膜区域可见出血|视网膜渗出少许出血|视网膜棉绒斑及出血|视网膜.方意思出血|视网膜大量斑点出血|视网膜水肿.状出血|视网膜暗红色灶出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜激光斑间或出血|视网膜偶见MA及出血|视网膜下渗出点状出血|视网膜散在MA及出血|视网膜前可见.状出血|视网膜后极部散在出血|视网膜大片网膜前出血|视网膜散在数小片出血|视网膜.量硬渗伴出血|视网膜较多MA及出血|视网膜散在点片状出血|视网膜火焰状片状出血|视网膜微血管瘤.出血|视网膜.下.范围出血|视网膜水肿斑片状出血|视网膜.量斑点状出血|视网膜广泛火焰状出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜激光斑间或出血|视网膜偶见MA及出血|视网膜下渗出点状出血|视网膜散在MA及出血|视网膜前可见.状出血|视网膜后极部散在出血|视网膜大片网膜前出血|视网膜散在数小片出血|视网膜.量硬渗伴出血|视网膜较多MA及出血|视网膜散在点片状出血|视网膜火焰状片状出血|视网膜微血管瘤.出血|视网膜.下.范围出血|视网膜水肿斑片状出血|视网膜.量斑点状出血|视网膜广泛火焰状出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜下渗出陈旧出血|视网膜.状点片状出血|视网膜.量放射状出血|视网膜平.方网膜出血|视网膜.量火焰状出血|视网膜似散在.状出血|视网膜水肿网膜下出血|视网膜可见白心样出血|视网膜.方.片状出血|视网膜偶见.片状出血|视网膜可见火焰状出血|视网膜散在斑片状出血|视网膜新生血管伴出血|视网膜.片状渗出出血|视网膜.片火焰状出血|视网膜.量点片状出血|视网膜可见MA及出血|视网膜可见MA小出血|视网膜大片放射状出血|视网膜偶见圆点状出血|视网膜前可见.量出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜下渗出陈旧出血|视网膜.状点片状出血|视网膜.量放射状出血|视网膜平.方网膜出血|视网膜.量火焰状出血|视网膜似散在.状出血|视网膜水肿网膜下出血|视网膜可见白心样出血|视网膜.方.片状出血|视网膜偶见.片状出血|视网膜可见火焰状出血|视网膜散在斑片状出血|视网膜新生血管伴出血|视网膜.片状渗出出血|视网膜.片火焰状出血|视网膜.量点片状出血|视网膜可见MA及出血|视网膜可见MA小出血|视网膜大片放射状出血|视网膜偶见圆点状出血|视网膜前可见.量出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜.可见.片出血|视网膜普遍斑点状出血|视网膜.上火焰状出血|视网膜圆形或.状出血|视网膜浓厚火焰状出血|视网膜有微血管瘤出血|视网膜MA及.片出血|视网膜.量MA及出血|视网膜散在棉绒斑出血|视网膜散在.片状出血|视网膜可见硬渗及出血|视网膜.量增殖膜出血|视网膜脱离后极部出血|视网膜可见白线及出血|视网膜.状火焰状出血|视网膜屏后极部见出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜.可见.片出血|视网膜普遍斑点状出血|视网膜.上火焰状出血|视网膜圆形或.状出血|视网膜浓厚火焰状出血|视网膜有微血管瘤出血|视网膜MA及.片出血|视网膜.量MA及出血|视网膜散在棉绒斑出血|视网膜散在.片状出血|视网膜可见硬渗及出血|视网膜.量增殖膜出血|视网膜脱离后极部出血|视网膜可见白线及出血|视网膜.状火焰状出血|视网膜屏后极部见出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜动脉细散在出血|视网膜平散在点状出血|视网膜散在数个小出血|视网膜.下方.状出血|视网膜可见一.状出血|视网膜.渗出增殖出血|视网膜散在..状出血|视网膜.量棉絮斑出血|视网膜.下方.片出血|视网膜仍可见少量出血|视网膜大片渗出见出血|视网膜.下方浓密出血|视网膜.侧数小片出血|视网膜四象限独处出血|视网膜广泛..状出血|视网膜偶见MA或出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜动脉细散在出血|视网膜平散在点状出血|视网膜散在数个小出血|视网膜.下方.状出血|视网膜可见一.状出血|视网膜.渗出增殖出血|视网膜散在..状出血|视网膜.量棉絮斑出血|视网膜.下方.片出血|视网膜仍可见少量出血|视网膜大片渗出见出血|视网膜.下方浓密出血|视网膜.侧数小片出血|视网膜四象限独处出血|视网膜广泛..状出血|视网膜偶见MA或出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜可见..状出血|视网膜可见MA或出血|视网膜散在渗出及出血|视网膜平见.点状出血|视网膜前膜见网膜出血|视网膜平见点片状出血|视网膜弥漫火焰状出血|视网膜颞上方.状出血|视网膜棉絮斑.状出血|视网膜可见小片状出血|视网膜片状红色斑出血|视网膜放射状广泛出血|视网膜.方.点状出血|视网膜仍可见散在出血|视网膜灰白伴.量出血|视网膜隐见一.状出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜可见..状出血|视网膜可见MA或出血|视网膜散在渗出及出血|视网膜平见.点状出血|视网膜前膜见网膜出血|视网膜平见点片状出血|视网膜弥漫火焰状出血|视网膜颞上方.状出血|视网膜棉絮斑.状出血|视网膜可见小片状出血|视网膜片状红色斑出血|视网膜放射状广泛出血|视网膜.方.点状出血|视网膜仍可见散在出血|视网膜灰白伴.量出血|视网膜隐见一.状出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜可见散在的出血|视网膜大片状浓密出血|视网膜片状及线状出血|视网膜多处MA及出血|视网膜是可见散在出血|视网膜大片网膜下出血|视网膜新生血管膜出血|视网膜隐见小片状出血|视网膜似有小片状出血|视网膜大量硬渗及出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜可见散在的出血|视网膜大片状浓密出血|视网膜片状及线状出血|视网膜多处MA及出血|视网膜是可见散在出血|视网膜大片网膜下出血|视网膜新生血管膜出血|视网膜隐见小片状出血|视网膜似有小片状出血|视网膜大量硬渗及出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜颞上方大片出血|视网膜隐见点片状出血|视网膜多处MA小出血|视网膜似有一小片出血|视网膜隐约见.量出血|视网膜陈旧小片状出血|视网膜较多点片状出血|视网膜血管前小片出血|视网膜隐见MA及出血|视网膜血管旁线状出血|视网膜前可见浓稠出血|视网膜大量渗出及出血|视网膜.侧半广泛出血|视网膜多个点片状出血|视网膜水肿及散在出血|视网膜可见小片.出血|视网膜小血管瘤小出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜颞上方大片出血|视网膜隐见点片状出血|视网膜多处MA小出血|视网膜似有一小片出血|视网膜隐约见.量出血|视网膜陈旧小片状出血|视网膜较多点片状出血|视网膜血管前小片出血|视网膜隐见MA及出血|视网膜血管旁线状出血|视网膜前可见浓稠出血|视网膜大量渗出及出血|视网膜.侧半广泛出血|视网膜多个点片状出血|视网膜水肿及散在出血|视网膜可见小片.出血|视网膜小血管瘤小出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜疑似小片状出血|视网膜小片裂片状出血|视网膜密集点片状出血|视网膜下大片渗出出血|视网膜可见软渗及出血|视网膜各象限.量出血|视网膜平可疑网膜出血|视网膜渗出网膜前出血|视网膜各象限可见出血|视网膜可见硬渗和出血|视网膜散在点片样出血|视网膜血管旁.状出血|视网膜大片棉绒斑出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜疑似小片状出血|视网膜小片裂片状出血|视网膜密集点片状出血|视网膜下大片渗出出血|视网膜可见软渗及出血|视网膜各象限.量出血|视网膜平可疑网膜出血|视网膜渗出网膜前出血|视网膜各象限可见出血|视网膜可见硬渗和出血|视网膜散在点片样出血|视网膜血管旁.状出血|视网膜大片棉绒斑出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                        # 视盘-视网膜出血
                    elif len(re.compile('视盘周及网膜.量出血|视盘及盘周网膜出血|视盘及网膜.量出血|视盘及.方视网膜出血|视网膜及视盘前出血|视盘表面及视网膜出血|视盘-视网膜出血|视盘-网膜出血|视盘及视网膜出血|视盘及网膜出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_视盘出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视盘周及网膜.量出血|视盘及盘周网膜出血|视盘及网膜.量出血|视盘及.方视网膜出血|视网膜及视盘前出血|视盘表面及视网膜出血|视盘-视网膜出血|视盘-网膜出血|视盘及视网膜出血|视盘及网膜出血').sub('出血_视网膜出血;出血_视盘出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜及黄斑区.片出血|视网膜及黄斑区出血|视网膜及黄斑区.出血|视网膜及黄斑出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_视盘出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜及黄斑区.片出血|视网膜及黄斑区出血|视网膜及黄斑区.出血').sub('出血_视网膜出血;出血_黄斑出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                     # 玻璃体-视网膜出血
                    elif len(re.compile('玻璃体积血网膜出血|玻璃体混浊网膜出血|玻璃体积血网膜出血|网膜前及玻璃体内可见出血|视网膜前或玻璃体腔可见出血|视网膜及玻璃体出血|视网膜前及玻璃体出血|视网膜前及玻璃部分出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('玻璃体积血网膜出血|玻璃体混浊网膜出血|玻璃体积血网膜出血|网膜前及玻璃体内可见出血|视网膜前或玻璃体腔可见出血|视网膜及玻璃体出血|视网膜前及玻璃体出血|视网膜前及玻璃部分出血').sub('出血_视网膜出血;出血_玻璃体出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜血管旁大片出血|视网膜血管旁小片出血|视网膜动脉旁一小出血|视网膜静脉迂曲伴出血|视网膜动脉旁可见出血|视网膜动脉旁小片出血|视网膜血管白线出血|视网膜静脉迂曲出血|视网膜血管异常出血|视网膜血管白鞘出血|视网膜血管紊乱出血|视网膜静脉旁出血|视网膜动脉旁出血|视网膜血管下出血|视网膜血管有出血|视网膜动脉细出血|视网膜血管旁出血|视网膜静脉可见出血|视网膜血管闭塞出血|视网膜血管表面出血|视网膜血管扩张出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜血管旁大片出血|视网膜血管旁小片出血|视网膜动脉旁一小出血|视网膜静脉迂曲伴出血|视网膜动脉旁可见出血|视网膜动脉旁小片出血|视网膜血管白线出血|视网膜静脉迂曲出血|视网膜血管异常出血|视网膜血管白鞘出血|视网膜血管紊乱出血|视网膜静脉旁出血|视网膜动脉旁出血|视网膜血管下出血|视网膜血管有出血|视网膜动脉细出血|视网膜血管旁出血|视网膜静脉可见出血|视网膜血管闭塞出血|视网膜血管表面出血|视网膜血管扩张出血').sub('出血_视网膜血管出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜血管白线眼底出血|视网膜血管局部少量出血|视网膜血管增殖机化出血|视网膜.脉迂曲可见出血|视网膜上下.脉迂曲出血|视网膜.上方.脉旁出血|视网膜局限血管迂曲出血|视网膜血管弓旁少量出血|视网膜静脉闭塞片状出血|视网膜动脉呈白线状出血|视网膜血管迂曲扩张出血|视网膜血管白鞘大量出血|视网膜静脉区域可见出血|视网膜静脉迂曲少量出血|视网膜血管白线少量出血|视网膜血管白线大片出血|视网膜静脉迂曲散在出血|视网膜血管旁一小出血|视网膜动脉旁似有出血|视网膜血管白线伴出血|视网膜动脉分支旁出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜血管白线眼底出血|视网膜血管局部少量出血|视网膜血管增殖机化出血|视网膜.脉迂曲可见出血|视网膜上下.脉迂曲出血|视网膜.上方.脉旁出血|视网膜局限血管迂曲出血|视网膜血管弓旁少量出血|视网膜静脉闭塞片状出血|视网膜动脉呈白线状出血|视网膜血管迂曲扩张出血|视网膜血管白鞘大量出血|视网膜静脉区域可见出血|视网膜静脉迂曲少量出血|视网膜血管白线少量出血|视网膜血管白线大片出血|视网膜静脉迂曲散在出血|视网膜血管旁一小出血|视网膜动脉旁似有出血|视网膜血管白线伴出血|视网膜动脉分支旁出血').sub('出血_视网膜血管出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜.见散在.样出血|视网膜火焰状.片状出血|视网膜平散在..状出血|视网膜可见.处.状出血|视网膜..MA及.出血|视网膜可见..MA出血|视网膜.见微血管瘤出血|视网膜微血管瘤点状出血|视网膜弥漫水肿散在出血|视网膜较多明显MA出血|视网膜可见新生血管出血|视网膜可见大片深层出血|视网膜可见.量.状出血|视网膜隐约可见大量出血|视网膜弥漫性火焰状出血|视网膜可见散在片状出血|视网膜可见散在.片出血|处视网膜.见..MA出血|视网膜可见MA.片出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
                        if len(re.compile('视网膜未见(.*)出血').findall(str_i_list[j]))>0:
                            value_list.append(str_i_list[j]) 
                        else:
#                             print(str_i_list[j])
                            str_i_list[j] = re.compile('视网膜.见散在.样出血|视网膜火焰状.片状出血|视网膜平散在..状出血|视网膜可见.处.状出血|视网膜..MA及.出血|视网膜可见..MA出血|视网膜.见微血管瘤出血|视网膜微血管瘤点状出血|视网膜弥漫水肿散在出血|视网膜较多明显MA出血|视网膜可见新生血管出血|视网膜可见大片深层出血|视网膜可见.量.状出血|视网膜隐约可见大量出血|视网膜弥漫性火焰状出血|视网膜可见散在片状出血|视网膜可见散在.片出血|处视网膜.见..MA出血|视网膜可见MA.片出血').sub('出血_视网膜出血',str_i_list[j])
                            value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜可见棉绒斑及出血|视网膜血管闭塞.状出血|视网膜水肿可见.量出血|视网膜散在微血管瘤出血|视网膜可见多发.状出血|视网膜.半可见.状出血|视网膜广泛散在MA出血|视网膜可见MA.状出血|视网膜.量微血管瘤出血|视网膜血管紊乱.见出血|视网膜水肿眼底散在出血|视网膜可见大片浓密出血|视网膜可见.状.状出血|视网膜微血管瘤渗出出血|视网膜.侧可疑.片出血|视网膜.方局限浓密出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜可见棉绒斑及出血|视网膜血管闭塞.状出血|视网膜水肿可见.量出血|视网膜散在微血管瘤出血|视网膜可见多发.状出血|视网膜.半可见.状出血|视网膜广泛散在MA出血|视网膜可见MA.状出血|视网膜.量微血管瘤出血|视网膜血管紊乱.见出血|视网膜水肿眼底散在出血|视网膜可见大片浓密出血|视网膜可见.状.状出血|视网膜微血管瘤渗出出血|视网膜.侧可疑.片出血|视网膜.方局限浓密出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜后极.片渗出出血|视网膜.片萎缩可见出血|视网膜点片状火焰状出血|视网膜可见一小片状出血|视网膜.片.性渗出出血|视网膜微血管瘤.状出血|视网膜可见点状零星出血|视网膜可见多发.片出血|视网膜前增殖膜散在出血|视网膜隐约可见MA出血|视网膜可见散在分布出血|视网膜.上方.片状出血|视网膜血管紊乱.量出血|网膜下出血|视网膜.性渗出.片出血|视网膜.量棉绒斑及出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜后极.片渗出出血|视网膜.片萎缩可见出血|视网膜点片状火焰状出血|视网膜可见一小片状出血|视网膜.片.性渗出出血|视网膜微血管瘤.状出血|视网膜可见点状零星出血|视网膜可见多发.片出血|视网膜前增殖膜散在出血|视网膜隐约可见MA出血|视网膜可见散在分布出血|视网膜.上方.片状出血|视网膜血管紊乱.量出血|网膜下出血|视网膜.性渗出.片出血|视网膜.量棉绒斑及出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜可点状渗出及出血|视网膜.方可见散在出血|视网膜可见.片病灶出血|视网膜色素变动散在出血|视网膜.下方见.状出血|视网膜.侧隐见..出血|视网膜散在.状.状出血|视网膜可见.层.状出血|视网膜可见棉絮斑及出血|视网膜.上可见.量出血|视网膜少许.状.状出血|视网膜可见散在数个出血|视网膜可见.片.层出血|视网膜.上分支可见出血|视网膜水肿似有.片出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
                        print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜可点状渗出及出血|视网膜.方可见散在出血|视网膜可见.片病灶出血|视网膜色素变动散在出血|视网膜.下方见.状出血|视网膜.侧隐见..出血|视网膜散在.状.状出血|视网膜可见.层.状出血|视网膜可见棉絮斑及出血|视网膜.上可见.量出血|视网膜少许.状.状出血|视网膜可见散在数个出血|视网膜可见.片.层出血|视网膜.上分支可见出血|视网膜水肿似有.片出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('视网膜血管旁多.片出血|视网膜大片增殖膜及出血|视网膜可见浓厚深层出血|视网膜颞下方大片状出血|视网膜小血管瘤.量出血|视网膜水肿灰白散在出血|视网膜水肿周围.个出血|视网膜血管旁.片状出血|视网膜四个象限.量出血|视网膜水肿周围.片出血|视网膜微较多血管瘤出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视网膜血管旁多.片出血|视网膜大片增殖膜及出血|视网膜可见浓厚深层出血|视网膜颞下方大片状出血|视网膜小血管瘤.量出血|视网膜水肿灰白散在出血|视网膜水肿周围.个出血|视网膜血管旁.片状出血|视网膜四个象限.量出血|视网膜水肿周围.片出血|视网膜微较多血管瘤出血').sub('出血_视网膜出血',str_i_list[j])
                        value_list.append(str_i_list[j]) 
                    elif len(re.compile('网膜散在出血|视盘.方网膜前出血|视盘颞.方网膜出血|视盘旁网膜线状出血|视盘鼻.方网膜出血|视盘后极部网膜出血|视盘网膜.形出血|视盘.方网膜出血|视盘颞侧网膜出血|视盘色.网膜出血|视盘鼻侧网膜出血|视盘水肿网膜出血|视盘周围网膜出血|视盘网膜大片出血|网膜大片出血|网膜陈旧出血|视盘网膜大量出血|视盘色.网膜出血|网膜见出血|网膜.见出血|网膜..见出血|网膜...见出血|视盘下网膜出血|视盘鼻下网膜出血|视盘下方网膜出血').findall(str_i_list[j]))>0 :  # 出血_视网膜出血;出血_玻璃体出血
                        if len(re.compile('未见出血').findall(str_i_list[j]))>0:
                            value_list.append(str_i_list[j]) 
                        else:
#                             print(str_i_list[j])
                            str_i_list[j] = re.compile('网膜散在出血|视盘.方网膜前出血|视盘颞.方网膜出血|视盘旁网膜线状出血|视盘鼻.方网膜出血|视盘后极部网膜出血|视盘网膜.形出血|视盘.方网膜出血|视盘颞侧网膜出血|视盘色.网膜出血|视盘鼻侧网膜出血|视盘水肿网膜出血|视盘周围网膜出血|视盘网膜大片出血|网膜大片出血|网膜陈旧出血|视盘网膜大量出血|视盘色.网膜出血|网膜见出血|网膜.见出血|网膜..见出血|网膜...见出血|视盘下网膜出血|视盘鼻下网膜出血|视盘下方网膜出血').sub('出血_视网膜出血',str_i_list[j])
                            value_list.append(str_i_list[j]) 
                    elif len(re.compile('后极部出血|后极部.出血|后极部..出血|后极部可见..出血|后极部...出血|后极部....出血|后极部.....出血|后极部......出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:  # 出血_视网膜出血;出血_玻璃体出血
                        if len(re.compile('未见出血').findall(str_i_list[j]))>0 :
                            value_list.append(str_i_list[j]) 
                        else:
                            print(str_i_list[j])
                            str_i_list[j] = re.compile('后极部出血|后极部.出血|后极部..出血|后极部可见..出血|后极部...出血|后极部....出血|后极部.....出血|后极部......出血').sub('出血_视网膜出血',str_i_list[j])
                            value_list.append(str_i_list[j]) 
                    else:
                        value_list.append(str_i_list[j])
                else:
                    value_list.append(str_i_list[j])
            df.at[i,'影描_gai'] = '!'.join(value_list[g] for g in range(len(value_list)))
#             print("==="+df.at[i,'影描_gai']) 
        
#     check_df_jb_0 = df[df['影描_gai'].str.contains('模糊')] 
#     check_df_jb = check_df_jb_0[check_df_jb_0['影描_gai'].str.contains('视盘.边缘模糊|视盘边缘.模糊|视盘.边界模糊|视盘边界.模糊|视盘界明显模糊|视盘.方界模糊|视盘色.界模糊|视盘充血界模糊|视盘边界轻模糊|视盘边界略模糊|视盘边缘模糊|视盘边色模糊|视盘边清模糊|视盘边境模糊|视盘边际模糊|视盘边清模糊|视盘图像.模糊|视盘-黄斑模糊|视盘区影像模糊|视盘前轻度模糊|视盘-眼底模糊|视盘颞侧.模糊|视盘鼻侧.模糊|视盘局限性模糊|视盘色淡余模糊|视盘.方稍模糊|视盘视网膜模糊|视盘前上方模糊')]  #特殊：右视神经病变待排青光眼?——因此有排青标签，易陷入死循环
#     if check_df_jb.shape[0] > 0:
#         print("剩余{}条数据未转化为".format(check_df_jb.shape[0]))
#         print(check_df_jb[['影像描述','影描_gai']])
#         normal_cd_fuc(df)
#     else:
#         print('转化结束！！！')         

In [ ]:
for i in range(5):
    normal_cd_fuc(df_ym)

In [ ]:
df_ym[df_ym['影描_gai'].str.contains('出血_(.*)\+出血_(.*)',re.I)]['影描_gai'] 

In [ ]:
####定义视盘出血——名称统一
def zero_point_func(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        else:
            str_tempt = re.split(r'!', str_i)
            str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
            value_list = []
            for j in range(len(str_i_list)):
                if len(re.compile('出血').findall(str_i_list[j]))>0 and len(re.compile('未见明显MA及上述病灶外的出血|未见明显异常..出血|未见明显MA及出血|未见明确MA及出血|未见明显MA出血|未见明显渗出及出血|无明显出血渗出|未见出血|未见..出血|未见渗出及出血|未见大出血|未见MA及出血点').findall(str_i_list[j]))==0:
                    if len(re.compile('视乳头..出血|视盘..出血|视乳头.出血|视盘.出血|视盘出血|视乳头出血|盘沿出血|盘周出血|盘缘出血|盘周可见出血|盘沿可见出血|盘缘可见出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视乳头..出血|视盘..出血|视乳头.出血|视盘.出血|视盘出血|视乳头出血|盘沿出血|盘周出血|盘缘出血|盘周可见出血|盘沿可见出血|盘缘可见出血').sub('出血_视盘出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视盘上.性出血|视盘上下方出血|视盘水肿及出血|视盘.上方出血|视盘鼻.方出血|视盘.下方出血|视盘颞.方出血|视盘前.片出血|视盘火焰.出血|视盘中央似出血|视盘血管袢出血|视盘周围有出血|视盘旁.量出血|视盘后极部出血|视盘.侧.出血|视盘旁网膜出血|视盘.线状出血|视盘上.状出血|视盘前.状出血|视盘.片状出血|视盘.可.出血|视盘及盘周出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视盘上.性出血|视盘上下方出血|视盘水肿及出血|视盘.上方出血|视盘鼻.方出血|视盘.下方出血|视盘颞.方出血|视盘前.片出血|视盘火焰.出血|视盘中央似出血|视盘血管袢出血|视盘周围有出血|视盘旁.量出血|视盘后极部出血|视盘.侧.出血|视盘旁网膜出血|视盘.线状出血|视盘上.状出血|视盘前.状出血|视盘.片状出血|视盘.可.出血|视盘及盘周出血').sub('出血_视盘出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视盘前似有出血|视盘肿胀伴出血|视盘.方见出血|视盘旁散在出血|视盘旁.片出血|视盘.一.出血|视盘可疑.出血|视盘.上有出血|视盘水肿伴出血|视盘.方.出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视盘前似有出血|视盘肿胀伴出血|视盘.方见出血|视盘旁散在出血|视盘旁.片出血|视盘.一.出血|视盘可疑.出血|视盘.上有出血|视盘水肿伴出血|视盘.方.出血').sub('出血_视盘出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视盘.方似有出血|视盘.方可疑出血|视盘彼此.量出血|视盘.方.状出血|视盘充血.量出血|视盘周.似有出血|视盘.方一.出血|视盘.火焰.出血|视盘表面.量出血|视盘水肿周.出血|视盘.下线状出血|视盘颞..状出血|视盘.侧疑似出血|视盘鼻..片出血|视盘鼻..装出血|视盘周..片出血|视盘周.可见出血|视盘.方.片出血|视盘鼻.可.出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视盘.方似有出血|视盘.方可疑出血|视盘彼此.量出血|视盘.方.状出血|视盘充血.量出血|视盘周.似有出血|视盘.方一.出血|视盘.火焰.出血|视盘表面.量出血|视盘水肿周.出血|视盘.下线状出血|视盘颞..状出血|视盘.侧疑似出血|视盘鼻..片出血|视盘鼻..装出血|视盘周..片出血|视盘周.可见出血|视盘.方.片出血|视盘鼻.可.出血').sub('出血_视盘出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视盘颞.可见出血|视盘周围可疑出血|视盘前房可见出血|视盘鼻侧偶见出血|视盘表面可.出血|视盘.侧.样出血|视盘周边.量出血|视盘.侧渗出出血|视盘鼻.渗出出血|视盘水肿现状出血|视盘鼻侧.状出血|视盘.方.形出血|视盘.下.形出血|视盘色.可见出血|视盘充血水肿出血|视盘.方可.出血|视盘.侧可.出血|视盘颞上一小出血|视盘鼻侧.状出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视盘颞.可见出血|视盘周围可疑出血|视盘前房可见出血|视盘鼻侧偶见出血|视盘表面可.出血|视盘.侧.样出血|视盘周边.量出血|视盘.侧渗出出血|视盘鼻.渗出出血|视盘水肿现状出血|视盘鼻侧.状出血|视盘.方.形出血|视盘.下.形出血|视盘色.可见出血|视盘充血水肿出血|视盘.方可.出血|视盘.侧可.出血|视盘颞上一小出血|视盘鼻侧.状出血').sub('出血_视盘出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视盘前.小片出血|视盘表面.状出血|视盘水肿.样出血|视盘水肿线.出血|视盘颞侧.片出血|视盘.方线.出血|视盘.方少许出血|视盘颞侧.片出血|视盘..局限出血|视盘局部.状出血|视盘.侧小片出血|视盘可见.状出血|视盘颞.可疑出血|视盘鼻.散在出血|视盘水肿充血出血|视盘.方水肿出血|视盘旁..状出血|视盘鼻..状出血|视盘.方盘缘出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视盘前.小片出血|视盘表面.状出血|视盘水肿.样出血|视盘水肿线.出血|视盘颞侧.片出血|视盘.方线.出血|视盘.方少许出血|视盘颞侧.片出血|视盘..局限出血|视盘局部.状出血|视盘.侧小片出血|视盘可见.状出血|视盘颞.可疑出血|视盘鼻.散在出血|视盘水肿充血出血|视盘.方水肿出血|视盘旁..状出血|视盘鼻..状出血|视盘.方盘缘出血').sub('出血_视盘出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视盘鼻侧似有出血|视盘可疑.状出血|视盘颞.线样出血|视盘表面.片出血|视盘鼻下色素出血|视盘鼻侧.量出血|视盘下半.片出血|视盘.方散在出血|视盘.方数个出血|视盘.方偶见出血|视盘.方原.出血|视盘.缘可疑出血|视盘颞下线样出血|视盘.方鼻侧出血|视盘鼻侧一小出血|视盘颞侧一小出血|视盘周围大量出血|视盘水肿.量出血|视盘边界不清出血|视盘色可网膜出血|视盘水肿.方出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视盘鼻侧似有出血|视盘可疑.状出血|视盘颞.线样出血|视盘表面.片出血|视盘鼻下色素出血|视盘鼻侧.量出血|视盘下半.片出血|视盘.方散在出血|视盘.方数个出血|视盘.方偶见出血|视盘.方原.出血|视盘.缘可疑出血|视盘颞下线样出血|视盘.方鼻侧出血|视盘鼻侧一小出血|视盘颞侧一小出血|视盘周围大量出血|视盘水肿.量出血|视盘边界不清出血|视盘色可网膜出血|视盘水肿.方出血').sub('出血_视盘出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视盘旁可见小出血|视盘上方疑似出血|视盘.方少量出血|视盘可疑.片出血|视盘色白少量出血|视盘水肿鼻下出血|视盘表面疑似出血|视盘表面线样出血|视盘.方.层出血|视盘颞上小片出血|视盘可见少量出血|视盘色淡散在出血|视盘颞侧细小出血|视盘正常隐约出血|视盘鼻上偶见出血|视盘上下点片出血|视盘周围线状出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视盘旁可见小出血|视盘上方疑似出血|视盘.方少量出血|视盘可疑.片出血|视盘色白少量出血|视盘水肿鼻下出血|视盘表面疑似出血|视盘表面线样出血|视盘.方.层出血|视盘颞上小片出血|视盘可见少量出血|视盘色淡散在出血|视盘颞侧细小出血|视盘正常隐约出血|视盘鼻上偶见出血|视盘上下点片出血|视盘周围线状出血').sub('出血_视盘出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视盘边缘.小片出血|视盘盘.上陈旧出血|视盘周围可疑.出血|视盘.侧见.状出血|视盘.侧可见.出血|视盘颞.可见.出血|视盘.方.片状出血|视盘.方.小片出血|视盘..方散在出血|视盘为中心广泛出血|视盘..方疑似出血|视盘处可见.状出血|视盘.下方.片出血|视盘.方火焰.出血|视盘水肿伴.状出血|视盘..方可见出血|视盘旁可见散在出血|视盘.下小.状出血|视盘.上方.状出血|视盘.侧各一小出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视盘边缘.小片出血|视盘盘.上陈旧出血|视盘周围可疑.出血|视盘.侧见.状出血|视盘.侧可见.出血|视盘颞.可见.出血|视盘.方.片状出血|视盘.方.小片出血|视盘..方散在出血|视盘为中心广泛出血|视盘..方疑似出血|视盘处可见.状出血|视盘.下方.片出血|视盘.方火焰.出血|视盘水肿伴.状出血|视盘..方可见出血|视盘旁可见散在出血|视盘.下小.状出血|视盘.上方.状出血|视盘.侧各一小出血').sub('出血_视盘出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视盘.侧一.状出血|视盘旁散在线状出血|视盘颞.方.量出血|视盘水肿小.样出血|视盘..方一下出血|视盘..方.状出血|视盘盘沿窄.状出血|视盘.下.状小出血|视盘..方小片出血|视盘..方小片出血|视盘上方各一片出血|视盘.侧数小片出血|视盘.方及.侧出血|视盘鼻侧斑片状出血|视盘边界欠清伴出血|视盘颞.方可疑出血|视盘边缘及我们出血|视盘.方见渗出出血|视盘.侧一小片出血|视盘表面火焰.出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视盘.侧一.状出血|视盘旁散在线状出血|视盘颞.方.量出血|视盘水肿小.样出血|视盘..方一下出血|视盘..方.状出血|视盘盘沿窄.状出血|视盘.下.状小出血|视盘..方小片出血|视盘..方小片出血|视盘上方各一片出血|视盘.侧数小片出血|视盘.方及.侧出血|视盘鼻侧斑片状出血|视盘边界欠清伴出血|视盘颞.方可疑出血|视盘边缘及我们出血|视盘.方见渗出出血|视盘.侧一小片出血|视盘表面火焰.出血').sub('出血_视盘出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视盘颞.方.片出血|视盘旁颞侧.片出血|视盘色可散在小出血|视盘.侧一条形出血|视盘.方零星.出血|视盘鼻侧盘沿小出血|视盘上有火焰.出血|视盘水肿放射.出血|视盘旁颞.线状出血|视盘边界不清伴出血|视盘旁可见.状出血|视盘周围火焰.出血|视盘鼻侧似.状出血|视盘.方血管前出血|视盘颞.方陈旧出血|视盘表面一.状出血|视盘鼻.可见一出血|视盘下方可疑小出血|视盘周围及.方出血|视盘上可见.状出血|视盘下方见.状出血|视盘..方大片火焰.出血|视盘界欠清可.出血|视盘水肿拌少量出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视盘颞.方.片出血|视盘旁颞侧.片出血|视盘色可散在小出血|视盘.侧一条形出血|视盘.方零星.出血|视盘鼻侧盘沿小出血|视盘上有火焰.出血|视盘水肿放射.出血|视盘旁颞.线状出血|视盘边界不清伴出血|视盘旁可见.状出血|视盘周围火焰.出血|视盘鼻侧似.状出血|视盘.方血管前出血|视盘颞.方陈旧出血|视盘表面一.状出血|视盘鼻.可见一出血|视盘下方可疑小出血|视盘周围及.方出血|视盘上可见.状出血|视盘下方见.状出血|视盘..方大片火焰.出血|视盘界欠清可.出血|视盘水肿拌少量出血').sub('出血_视盘出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视盘水肿伴少量出血|视盘表面小.状出血|视盘周边小.状出血|视盘色淡水肿有出血|视盘后极部隐见出血|视盘.方数个小出血|视盘不清鼻上方出血|视盘界模糊少量出血|视盘鼻.方一小出血|视盘鼻.见.状出血|视盘区可见片状出血|视盘表面血管襻出血|视盘上可见小片出血|视盘边界清似有出血|视盘.方小.状出血|视盘鼻.方可以出血|视盘旁新生血管出血|视盘下方元点状出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视盘水肿伴少量出血|视盘表面小.状出血|视盘周边小.状出血|视盘色淡水肿有出血|视盘后极部隐见出血|视盘.方数个小出血|视盘不清鼻上方出血|视盘界模糊少量出血|视盘鼻.方一小出血|视盘鼻.见.状出血|视盘区可见片状出血|视盘表面血管襻出血|视盘上可见小片出血|视盘边界清似有出血|视盘.方小.状出血|视盘鼻.方可以出血|视盘旁新生血管出血|视盘下方元点状出血').sub('出血_视盘出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视盘鼻下似.状出血|视盘颞.方隐见出血|视盘颞.方似有出血|视盘水肿其周有出血|视盘界不清.方出血|盘鼻侧零星小出血|视盘周可见.形出血|视盘明显水肿伴出血|视盘颞上方一小出血|视盘.方似有小出血|视盘鼻.方深层出血|视盘鼻.MA或出血|视盘色淡红偶见出血|视盘上方血管袢出血|视盘边缘小.状出血|视盘前可见.状出血|视盘鼻侧小.状出血|视盘.状火焰.出血|视盘水肿小.状出血|视盘界不清水肿出血|视盘.方盘沿窄出血|视盘旁.处圆形出血|视盘色可后极部出血|视盘界模糊盘内出血|视盘颞.方一小出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('视盘鼻下似.状出血|视盘颞.方隐见出血|视盘颞.方似有出血|视盘水肿其周有出血|视盘界不清.方出血|盘鼻侧零星小出血|视盘周可见.形出血|视盘明显水肿伴出血|视盘颞上方一小出血|视盘.方似有小出血|视盘鼻.方深层出血|视盘鼻.MA或出血|视盘色淡红偶见出血|视盘上方血管袢出血|视盘边缘小.状出血|视盘前可见.状出血|视盘鼻侧小.状出血|视盘.状火焰.出血|视盘水肿小.状出血|视盘界不清水肿出血|视盘.方盘沿窄出血|视盘旁.处圆形出血|视盘色可后极部出血|视盘界模糊盘内出血|视盘颞.方一小出血').sub('出血_视盘出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('视盘...出血|视盘....出血|视盘.....出血|视盘......出血|视盘.......出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
                        if len(re.compile('黄斑|眼底|玻璃体|未见大出血|未见渗出及出血').findall(str_i_list[j]))>0:
                            value_list.append(str_i_list[j])
                        else:
                            print(str_i_list[j])
                            str_i_list[j] = re.compile('视盘...出血|视盘....出血|视盘.....出血|视盘......出血|视盘.......出血').sub('出血_视盘出血',str_i_list[j])
                            value_list.append(str_i_list[j])
                    else:
                        value_list.append(str_i_list[j])
                else:
                    value_list.append(str_i_list[j])
            df.at[i,'影描_gai'] = '!'.join(value_list[g] for g in range(len(value_list)))
#             print("==="+df.at[i,'影描_gai']) 
                                      
#     check_df_jb_0 = df[df['影描_gai'].str.contains('片状出血|火焰状出血|出血斑|团状出血|放射状出血')] 
#     check_df_jb = check_df_jb_0[~check_df_jb_0['影描_gai'].str.contains('出血_点状出血|出血_火焰状出血|出血_出血斑|出血_团状出血|出血_放射状出血')]  #特殊：右视神经病变待排青光眼?——因此有排青标签，易陷入死循环
#     if check_df_jb.shape[0] == 0:
#         print('转化结束！！！')  
#     else:
#         print("剩余{}条数据未转化为".format(check_df_jb.shape[0]))
#         print(check_df_jb[['影像描述','影描_gai']])
#         normal_cd_fuc(df)

In [ ]:
for i in range(5):
    zero_point_func(df_ym)

In [ ]:
####定义黄斑出血——名称统一
def hb_point_func(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        else:
            str_tempt = re.split(r'!', str_i)
            str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
            value_list = []
            for j in range(len(str_i_list)):
                if len(re.compile('出血').findall(str_i_list[j]))>0 and len(re.compile('未见明显MA及上述病灶外的出血|未见明显异常..出血|未见明显MA及出血|未见明确MA及出血|未见明显MA出血|未见明显渗出及出血|无明显出血渗出|未见出血|未见..出血|未见渗出及出血|未见大出血|未见MA及出血点').findall(str_i_list[j]))==0:
                    if len(re.compile('黄斑出血|黄斑.出血|黄斑..出血|视网膜前膜出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
#                         print(str_i_list[j])
                        str_i_list[j] = re.compile('黄斑出血|黄斑.出血|黄斑..出血|视网膜前膜出血').sub('出血_黄斑出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('黄斑...出血|黄斑....出血|黄斑.....出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
                        if len(re.compile('渗出血管').findall(str_i_list[j]))>0:
                            value_list.append(str_i_list[j])
                        else:
#                             print(str_i_list[j])
                            str_i_list[j] = re.compile('黄斑...出血|黄斑....出血|黄斑.....出血').sub('出血_黄斑出血',str_i_list[j])
                            value_list.append(str_i_list[j])
                    elif len(re.compile('黄斑......出血|黄斑.......出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
                        if len(re.compile('网膜').findall(str_i_list[j]))>0:
                            value_list.append(str_i_list[j])
                        else:
                            print(str_i_list[j])
                            str_i_list[j] = re.compile('黄斑......出血|黄斑.......出血').sub('出血_黄斑出血',str_i_list[j])
                            value_list.append(str_i_list[j])
                    else:
                        value_list.append(str_i_list[j])
                else:
                    value_list.append(str_i_list[j])
            df.at[i,'影描_gai'] = '!'.join(value_list[g] for g in range(len(value_list)))
#             print("==="+df.at[i,'影描_gai']) 
                                      
#     check_df_jb_0 = df[df['影描_gai'].str.contains('片状出血|火焰状出血|出血斑|团状出血|放射状出血')] 
#     check_df_jb = check_df_jb_0[~check_df_jb_0['影描_gai'].str.contains('出血_点状出血|出血_火焰状出血|出血_出血斑|出血_团状出血|出血_放射状出血')]  #特殊：右视神经病变待排青光眼?——因此有排青标签，易陷入死循环
#     if check_df_jb.shape[0] == 0:
#         print('转化结束！！！')  
#     else:
#         print("剩余{}条数据未转化为".format(check_df_jb.shape[0]))
#         print(check_df_jb[['影像描述','影描_gai']])
#         normal_cd_fuc(df)

In [ ]:
for i in range(5):
    hb_point_func(df_ym)

In [ ]:
####定义玻璃体出血——名称统一
def blt_point_func(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        else: 
            str_tempt = re.split(r'!', str_i)
            str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
            value_list = []
            for j in range(len(str_i_list)):
                if len(re.compile('出血').findall(str_i_list[j]))>0 and len(re.compile('未见明显MA及上述病灶外的出血|未见明显异常..出血|未见明显MA及出血|未见明确MA及出血|未见明显MA出血|未见明显渗出及出血|无明显出血渗出|未见出血|未见..出血|未见渗出及出血|未见大出血|未见MA及出血点').findall(str_i_list[j]))==0:
                    if len(re.compile('玻璃体眼底出血|玻璃体与眼底出血|玻璃体及眼底可见大片出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
                        print(str_i_list[j])
                        str_i_list[j] = re.compile('玻璃体眼底出血|玻璃体与眼底出血|玻璃体及眼底可见大片出血').sub('出血_玻璃体出血;出血_眼底出血',str_i_list[j])
                        value_list.append(str_i_list[j])
                    elif len(re.compile('玻璃体出血|玻璃体.出血|玻璃体..出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
                        if len(re.compile('渗出血管|眼底出血').findall(str_i_list[j]))>0:
                            value_list.append(str_i_list[j])
                        else:
                            print(str_i_list[j])
                            str_i_list[j] = re.compile('玻璃体出血|玻璃体.出血|玻璃体..出血').sub('出血_玻璃体出血',str_i_list[j])
                            value_list.append(str_i_list[j])
                    elif len(re.compile('玻璃体...出血|玻璃体....出血|玻璃体.....出血|玻璃体......出血|玻璃体.......出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
                        if len(re.compile('眼底出血|网膜出血|眼底可见出血|网膜前出血|眼底隐见出血|网膜.状出血').findall(str_i_list[j]))>0:
                            value_list.append(str_i_list[j])
                        else:
                            print(str_i_list[j])
                            str_i_list[j] = re.compile('玻璃体...出血|玻璃体....出血|玻璃体.....出血|玻璃体......出血|玻璃体.......出血').sub('出血_玻璃体出血',str_i_list[j])
                            value_list.append(str_i_list[j])
                    else:
                        value_list.append(str_i_list[j])
                else:
                    value_list.append(str_i_list[j])
            df.at[i,'影描_gai'] = '!'.join(value_list[g] for g in range(len(value_list)))
#             print("==="+df.at[i,'影描_gai']) 
                                      
#     check_df_jb_0 = df[df['影描_gai'].str.contains('片状出血|火焰状出血|出血斑|团状出血|放射状出血')] 
#     check_df_jb = check_df_jb_0[~check_df_jb_0['影描_gai'].str.contains('出血_点状出血|出血_火焰状出血|出血_出血斑|出血_团状出血|出血_放射状出血')]  #特殊：右视神经病变待排青光眼?——因此有排青标签，易陷入死循环
#     if check_df_jb.shape[0] == 0:
#         print('转化结束！！！')  
#     else:
#         print("剩余{}条数据未转化为".format(check_df_jb.shape[0]))
#         print(check_df_jb[['影像描述','影描_gai']])
#         normal_cd_fuc(df)

In [ ]:
for i in range(5):
    blt_point_func(df_ym)

### 视网膜前膜——别称——特发性黄斑部前膜
膜视网膜前膜是一种与年龄相关的增生性疾病，表现为黄斑部视网膜前膜形成及其收缩导致的继发性改变

In [ ]:
####定义网膜.出血——名称统一
def wm_point_func(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        else:
            str_tempt = re.split(r'!', str_i)
            str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
            value_list = []
            for j in range(len(str_i_list)):
                if len(re.compile('出血').findall(str_i_list[j]))>0 and len(re.compile('未见明显MA及上述病灶外的出血|未见明显异常..出血|未见明显MA及出血|未见明确MA及出血|未见明显MA出血|未见明显渗出及出血|无明显出血渗出|未见出血|未见..出血|未见渗出及出血|未见大出血|未见MA及出血点').findall(str_i_list[j]))==0:
                    if len(re.compile('网膜出血|网膜.出血|网膜..出血|网膜...出血|网膜....出血|网膜.....出血|网膜......出血|网膜.......出血|视网膜........出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
                        if len(re.compile('视网膜出血|视盘及|网膜前膜|网膜无渗出及出血|网膜无出血|网膜未见渗出及出血|网膜未见MA及出血|网膜未见微血管瘤出血').findall(str_i_list[j]))>0:
                            value_list.append(str_i_list[j])
                        else:
                            print(str_i_list[j])
                            str_i_list[j] = re.compile('网膜出血|网膜.出血|网膜..出血|网膜...出血|网膜....出血|网膜.....出血|网膜......出血|网膜.......出血|视网膜........出血').sub('出血_视网膜出血',str_i_list[j])
                            value_list.append(str_i_list[j])
                    elif len(re.compile('网膜前膜出血|网膜前膜.出血|网膜前膜..出血|网膜前膜...出血|网膜前膜....出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
                        if len(re.compile('视网膜前膜出血|网膜前膜无渗出及出血|网膜前膜无出血|眼底出血').findall(str_i_list[j]))>0:
                            value_list.append(str_i_list[j])
                        else:
                            print(str_i_list[j])
                            str_i_list[j] = re.compile('网膜前膜出血|网膜前膜.出血|网膜前膜..出血|网膜前膜...出血|网膜前膜....出血').sub('出血_黄斑出血',str_i_list[j])
                            value_list.append(str_i_list[j])
                    elif len(re.compile('视网膜前膜出血|视网膜前膜.出血|视网膜前膜..出血|视网膜前膜...出血|视网膜前膜....出血|视网膜前膜.....出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
                        if len(re.compile('无渗出及出血|前膜无出血|眼底出血').findall(str_i_list[j]))>0:
                            value_list.append(str_i_list[j])
                        else:
                            print(str_i_list[j])
                            str_i_list[j] = re.compile('视网膜前膜出血|视网膜前膜.出血|视网膜前膜..出血|视网膜前膜...出血|视网膜前膜....出血|视网膜前膜.....出血').sub('出血_黄斑出血',str_i_list[j])
                            value_list.append(str_i_list[j])
                    else:
                        value_list.append(str_i_list[j])
                else:
                    value_list.append(str_i_list[j])
            df.at[i,'影描_gai'] = '!'.join(value_list[g] for g in range(len(value_list)))
#             print("==="+df.at[i,'影描_gai']) 

In [ ]:
for i in range(5):
    wm_point_func(df_ym)

In [ ]:
####定义眼底出血——名称统一
def yd_point_func(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i == 'nan':
            pass
        else:
            str_tempt = re.split(r'!', str_i)
            str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
            value_list = []
            for j in range(len(str_i_list)):
                if len(re.compile('出血').findall(str_i_list[j]))>0 and len(re.compile('未见明显MA及上述病灶外的出血|未见明显异常..出血|未见明显MA及出血|未见明确MA及出血|未见明显MA出血|未见明显渗出及出血|无明显出血渗出|未见出血|未见..出血|未见渗出及出血|未见大出血|未见MA及出血点').findall(str_i_list[j]))==0:
                    if len(re.compile('眼底出血|眼底.出血|眼底..出血|眼底...出血|眼底....出血|眼底.....出血|眼底......出血').findall(str_i_list[j]))>0 and len(re.compile('出血_').findall(str_i_list[j]))==0:
                        if len(re.compile('无渗出及出血|无出血|未见渗出及出血|未见MA及出血|未见微血管瘤出血|眼底不清').findall(str_i_list[j]))>0:
                            value_list.append(str_i_list[j])
                        else:
#                             print(str_i_list[j])
                            str_i_list[j] = re.compile('眼底出血|眼底.出血|眼底..出血|眼底...出血|眼底....出血|眼底.....出血|眼底......出血').sub('出血_眼底出血',str_i_list[j])
                            value_list.append(str_i_list[j])
                    elif len(re.compile('底出血|底.出血|底..出血|底...出血|底....出血').findall(str_i_list[j]))>0 and len(re.compile('眼底出血').findall(str_i_list[j]))==0:
                        if len(re.compile('无渗出及出血|无出血|未见渗出及出血|未见MA及出血|未见微血管瘤出血|眼底不清|出血_').findall(str_i_list[j]))>0:
                            value_list.append(str_i_list[j])
                        else:
                            print(str_i_list[j])
                            str_i_list[j] = re.compile('底出血|底.出血|底..出血|底...出血|底....出血').sub('出血_眼底出血',str_i_list[j])
                            value_list.append(str_i_list[j])
                    else:
                        value_list.append(str_i_list[j])
                else:
                    value_list.append(str_i_list[j])
            df.at[i,'影描_gai'] = '!'.join(value_list[g] for g in range(len(value_list)))
#             print("==="+df.at[i,'影描_gai']) 

In [ ]:
for i in range(5):
    yd_point_func(df_ym)

## 添加新标签

In [ ]:
df_ym.insert(df_ym.columns.get_loc('影描_gai')+1,'right_出血location',None)
df_ym.insert(df_ym.columns.get_loc('影描_gai')+1,'left_出血location',None)

In [ ]:
# 定义——主体化标签——统一的糖网可能情形列表
column_zu = ['出血']    # 仅匹配出血标签

In [ ]:
# 定义——主体化标签——统一的糖网可能情形列表
column_tw = ['出血_视网膜出血','出血_视网膜血管出血','出血_视盘出血','出血_黄斑出血','出血_玻璃体出血','出血_眼底出血',
             '出血_视网膜出血;出血_黄斑出血','出血_视网膜出血;出血_玻璃体出血','出血_视网膜出血;出血_视盘出血','出血_玻璃体出血;出血_眼底出血']

In [ ]:
# 添加出血的解析标签
def add_new_label_fuc01(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i=='nan':   # 排除str_i为”null“的项
            pass
        ## 未见明确出血、 未见明显出血
        elif len(re.compile('出血').findall(str_i))>0 and len(re.compile('未见明显MA及上述病灶外的出血|未见明显异常..出血|未见明显MA及出血|未见明确MA及出血|未见明显MA出血|未见明显渗出及出血|无明显出血渗出|未见出血|未见..出血|未见渗出及出血|未见大出血|未见MA及出血点').findall(str_i))==0:
            str_tempt = re.split(r'!', str_i)
            str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
            for j in range(len(column_zu)):
                column_zu_j = column_zu[j]
                column_zu_j_list = column_zu_j.split('_')
    #             print(column_zu_j_list)       
                for k in range(len(str_i_list)):
                    if (str.find(str_i_list[k],column_zu_j)!=-1) and (str.find(str_i_list[k],'双')!=-1):
        #                     print(str_i_list[k]) .join(str_i_new_list[m] for m in range(len(str_i_new_list)))
                        df.at[i,'left_出血location'] = ','.join(column_zu_j_list[g] for g in range(len(column_zu_j_list)))
                        df.at[i,'right_出血location'] = ','.join(column_zu_j_list[g] for g in range(len(column_zu_j_list)))
                    elif (str.find(str_i_list[k],column_zu_j)!=-1) and (str.find(str_i_list[k],'左')!=-1):
    #                     print(str_i_list[k])
                        df.at[i,'left_出血location'] = ','.join(column_zu_j_list[g] for g in range(len(column_zu_j_list)))
                    elif (str.find(str_i_list[k],column_zu_j)!=-1) and (str.find(str_i_list[k],'右')!=-1):
    #                     print(str_i_list[k])
                        df.at[i,'right_出血location'] = ','.join(column_zu_j_list[g] for g in range(len(column_zu_j_list)))
                    else:
                        pass
        else:
            pass

In [ ]:
add_new_label_fuc01(df_ym)

In [ ]:
# 添加出血_视网膜出血等的解析标签
def add_new_label_fuc02(df):
    for i in range(df.shape[0]):
        str_i = str(df.at[i,'影描_gai'])
        if str_i=='nan':   # 排除str_i为”null“的项
            pass
        else:
            str_tempt = re.split(r'!', str_i)
            str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
            for j in range(len(column_tw)):
                column_tw_j = column_tw[j]
                column_tw_j_list = column_tw_j.split('_')
    #             print(column_tw_j_list)       
                for k in range(len(str_i_list)):
                    if (str.find(str_i_list[k],column_tw_j)!=-1) and (str.find(str_i_list[k],'双')!=-1):
        #                     print(str_i_list[k]) .join(str_i_new_list[m] for m in range(len(str_i_new_list)))
                        df.at[i,'left_出血location'] = ','.join(column_tw_j_list[g] for g in range(len(column_tw_j_list)))
                        df.at[i,'right_出血location'] = ','.join(column_tw_j_list[g] for g in range(len(column_tw_j_list)))
                    elif (str.find(str_i_list[k],column_tw_j)!=-1) and (str.find(str_i_list[k],'左')!=-1):
    #                     print(str_i_list[k])
                        df.at[i,'left_出血location'] = ','.join(column_tw_j_list[g] for g in range(len(column_tw_j_list)))
                    elif (str.find(str_i_list[k],column_tw_j)!=-1) and (str.find(str_i_list[k],'右')!=-1):
    #                     print(str_i_list[k])
                        df.at[i,'right_出血location'] = ','.join(column_tw_j_list[g] for g in range(len(column_tw_j_list)))
                    else:
                        pass

In [ ]:
add_new_label_fuc02(df_ym)   # 调用添加新标签的函数

### 标签修正，单眼罹患多类出血location的疾病种类~

In [ ]:
# 定义——副的主体化标签——排除“黄斑病变”，避免删除函数涉及“黄斑病变”的删除
column_tw_fu = ['出血_视网膜出血','出血_视网膜血管出血','出血_视盘出血','出血_黄斑出血','出血_玻璃体出血','出血_眼底出血',
             '出血_视网膜出血;出血_黄斑出血','出血_视网膜出血;出血_玻璃体出血','出血_视网膜出血;出血_视盘出血','出血_玻璃体出血;出血_眼底出血']

In [ ]:
# 删除重复字符串函数
def delect_fuc(x):
    str_x = ['视网膜出血','视网膜血管出血','视盘出血','黄斑出血','玻璃体出血','眼底出血'] # 二级标签列表
    y = {}  # 设置查重字符串字典
    check_count = {}
    for i in range(len(str_x)):
        y[str_x[i]] = x.count(str_x[i])
#         print("重复字符串：{}".format(y))
        if y[str_x[i]] > 1:
            duplicate_start_x = [j.start() for j in re.finditer(str_x[i],x)]
#             print(duplicate_start_x)
            separate_list = [j.start() for j in re.finditer(';',x)]
#             print(separate_list)
            if separate_list[0] < duplicate_start_x[0]:  # 非重复标签开头
                index_start = [j for j in separate_list if j < duplicate_start_x[0]][-1]  # 起始索引——截取小于min重复词的；中，索引值最大的一个。
                print("删除字符串的起始索引：{}".format(index_start))
                index_end = [k for k in separate_list if k < duplicate_start_x[-1]][-1]   # 结束索引——截取小于max重复词；中，索引值最大的一个。
                print("删除字符串的结束索引：{}".format(index_end))   
                x = x[:index_start] + x[index_end:]
                print("删除多余字符之后的标签项——x：{}".format(x))
            else:  # 重复标签开头
                index_start = 0
                print("删除字符串的起始索引：{}".format(index_start))
                index_end = [k for k in separate_list if k < duplicate_start_x[-1]][-1]+1   # 结束索引——截取小于max重复词；中，索引值最大的一个。
                print("删除字符串的结束索引：{}".format(index_end))   
                x = x[:index_start] + x[index_end:]
                print("删除多余字符之后的标签项——x：{}".format(x))
        else:
            pass
    return x 

In [ ]:
# "必须含双，可能含左、右含多种疾病的打标"函数
def str_fix_fuc(a):
    str_tempt = re.split(r'!',a)
    str_i_list = list(filter(None,str_tempt))
    print(str_i_list)
    for j in range(len(str_i_list)):
        print(str_i_list[j])
        for i in range(len(column_tw_fu)):
            column_tw_fu_i_list = column_tw_fu[i].split('_')
            if len(re.compile(column_tw_fu[i]).findall(str_i_list[j])) > 0 and len(re.compile('双').findall(str_i_list[j])) > 0:
                r = l = ','.join(column_tw_fu_i_list[g] for g in range(len(column_tw_fu_i_list)))
                print("d============={}".format(r))
            elif len(re.compile(column_tw_fu[i]).findall(str_i_list[j])) > 0 and len(re.compile('右').findall(str_i_list[j])) > 0:
                d1 = ','.join(column_tw_fu_i_list[g] for g in range(len(column_tw_fu_i_list)))
                print(d1)  # 右后极部视网膜出血_片状出血 双底模糊!右后极部视网膜出血_片状出血!左底散在出血_出血斑及渗出斑部分玻璃体出血
                r = ';'.join([r,d1])
                print("r**************{}".format(r))
            elif len(re.compile(column_tw_fu[i]).findall(str_i_list[j])) > 0 and len(re.compile('左').findall(str_i_list[j])) > 0:
                d2 = ','.join(column_tw_fu_i_list[g] for g in range(len(column_tw_fu_i_list)))
#                 print(d2)
                l = ';'.join([l,d2])
                print("l~~~~~~~~~~~~~~~{}".format(l))
            else:
                pass
    r = delect_fuc(r)
    l = delect_fuc(l)
                
    return r,l

In [ ]:
# 处理“左、左”或“右、右”多种疾病的情形函数
def str_fix_single_fuc(a,mtch_qz):  # mtch_qz为匹配的前缀，a为匹配的str_n
    str_tempt = re.split(r'!',a)
    str_i_list = list(filter(None,str_tempt))
    print(str_i_list)
    l_list = []
    for j in range(len(str_i_list)):
        print(str_i_list[j])
        l_list_x = []   # 存放单一的“左黄斑病变_黄斑变性_AMD_湿性可能性大”项
        for i in range(len(column_tw_fu)):
            column_tw_fu_i_list = column_tw_fu[i].split('_')
            if len(re.compile(column_tw_fu[i]).findall(str_i_list[j])) > 0 and len(re.compile(mtch_qz).findall(str_i_list[j])) > 0:
                d = ','.join(column_tw_fu_i_list[g] for g in range(len(column_tw_fu_i_list)))
                l_list_x.append(d)
            else:
                pass
#         print("***********{}".format(l_list))
        l1 = ';'.join(i for i in l_list_x)
        l1 = delect_fuc(l1)
#         print("l1{}".format(l1))
        l_list.append(l1)
#         print("=============l{}".format(l_list))
    l = ';'.join( k for k in l_list )               
    return l

In [ ]:
# 修正每一眼罹患多种疾病的标签
def fix_new_label_fuc(df):
    for n in range(df.shape[0]):
        str_n = str(df.at[n,'影描_gai'])
        if str_n=='nan':   # 排除str_i为”null“的项
            pass
        elif len(re.compile('双可见出血_(.*)左(.*)出血_|双可见出血_(.*)右(.*)出血_',re.I).findall(str_n))>0:
            print(str_n)
            df.at[n,'right_出血location'],df.at[n,'left_出血location'] = str_fix_fuc(str_n)
            print(df.at[n,'right_出血location']+"__"+df.at[n,'left_出血location'])
        elif len(re.compile('左(.*)出血_(.*)左(.*)出血_').findall(str_n))>0:
            y_temp = str_n.split('!')
            index_list = [ i for i in range(len(y_temp)) if re.compile('出血_').findall(y_temp[i]) ]
            print(index_list)
            if len(index_list) < 2:
                pass
            else:
                tempt_a = len(y_temp[index_list[0]])
                tempt_b = len(y_temp[index_list[1]])
#                 print(tempt_a,tempt_b)
                if tempt_a < tempt_b:  # 寻找字符串短的为子集，长的为全集
                    min_index = y_temp[index_list[0]] 
                    max_index = y_temp[index_list[1]]
                else:
                    min_index = y_temp[index_list[1]]
                    max_index = y_temp[index_list[0]]
                print(min_index,max_index)
                if re.compile(min_index).findall(max_index):  # 包含子集，即左——左——疾病为包含关系
                    pass
                else:
                    df.at[n,'left_出血location'] = str_fix_single_fuc(str_n,'左')
        elif len(re.compile('右(.*)出血_(.*)右(.*)出血_').findall(str_n))>0:
            y_temp = str_n.split('!')
            index_list = [ i for i in range(len(y_temp)) if re.compile('出血_').findall(y_temp[i]) ]
            print(index_list)
            if len(index_list) < 2:
                pass
            else:
                tempt_a = len(y_temp[index_list[0]])
                tempt_b = len(y_temp[index_list[1]])
    #             print(tempt_a,tempt_b)
                if tempt_a < tempt_b:  # 寻找字符串短的为子集，长的为全集
                    min_index = y_temp[index_list[0]] 
                    max_index = y_temp[index_list[1]]
                else:
                    min_index = y_temp[index_list[1]]
                    max_index = y_temp[index_list[0]]
                print("min_index:{},max_index:{}".format(min_index,max_index))
                if re.compile(min_index).findall(max_index):  # 包含子集，即左——左——疾病为包含关系
                    pass
                else:
                    df.at[n,'right_出血location'] = str_fix_single_fuc(str_n,'右')
        else:
            pass

In [ ]:
fix_new_label_fuc(df_ym)

In [ ]:
# 校验标签无重复子集项
def check_duplicated_fuc(df,column_name):
    check_count = {}
    for i in range(df.shape[0]):
        str_i = str(df.at[i,column_name])
        if str_i=='nan':   # 排除str_i为”null“的项
            pass
        else:
            for m in range(len(column_tw_fu)):
                check_count[column_tw_fu[m]] = str_i.count(column_tw_fu[m])
    #             print("重复字符串：{}".format(check_count))
                if check_count[column_tw_fu[m]] > 1:
                    print(str_i)

In [ ]:
check_duplicated_fuc(df_ym,'left_出血location')
check_duplicated_fuc(df_ym,'right_出血location')

### '影描_gai'字段——含“出血”标签的数据容量

In [ ]:
a = df_ym[df_ym['影描_gai'].str.contains('出血')]

In [ ]:
a.shape[0]

In [ ]:
b = a[~a['影描_gai'].str.contains('未见明显MA及上述病灶外的出血|未见明显异常..出血|未见明显MA及出血|未见明确MA及出血|未见明显MA出血|未见明显渗出及出血|无明显出血渗出|未见出血|未见..出血|未见渗出及出血|未见大出血|未见MA及出血点')]

In [ ]:
# 最终含出血的样本容量
b.shape[0]

In [ ]:
print("'影描_gai'字段——含出血location——数据容量:{}".format(df_ym[df_ym['影描_gai'].str.contains('出血')].shape[0] ))  # 含青光眼的数据容量

In [ ]:
# 去除多余的；
for i in range(10):
    df_ym['left_出血location'].replace(to_replace=';;',value='',inplace=True)
    df_ym['right_出血location'].replace(to_replace=';;',value='',inplace=True)
    # 使用正则表达式替换——去掉开头的；
    df_ym['left_出血location'].replace(to_replace=r'^;',value='',regex=True,inplace=True)
    df_ym['right_出血location'].replace(to_replace=r'^;',value='',regex=True,inplace=True)
    # 使用正则表达式替换——去掉结尾的；
    df_ym['left_出血location'].replace(to_replace=r';$',value='',regex=True,inplace=True)
    df_ym['right_出血location'].replace(to_replace=r';$',value='',regex=True,inplace=True)

In [ ]:
# 去除顽固、多余的；
def delete_func(df,column_name):
    for j in range(10):
        for i in range(df.shape[0]):
            str_n = str(df.at[i,column_name])
            if str_n=='nan':
                pass
            else:
                if len(re.compile(';;').findall(str_n))>0:
#                     print(str_n)
                    df.at[i,column_name] = re.compile(';;').sub(';',str_n)
#                     print('__'+df_ym.at[i,column_name])
                elif len(re.compile('出血,视网膜出血;出血,视网膜出血').findall(str_n))>0:
#                     print(str_n)
                    df.at[i,column_name] = re.compile('出血,视网膜出血;出血,视网膜出血').sub('出血,视网膜出血',str_n)
#                     print('__'+df_ym.at[i,'right_出血location'])
                elif len(re.compile('出血,黄斑出血;出血,黄斑出血').findall(str_n))>0:
#                     print(str_n)
                    df.at[i,column_name] = re.compile('出血,黄斑出血;出血,黄斑出血').sub('出血,黄斑出血',str_n)
#                     print('__'+df_ym.at[i,column_name])
                elif len(re.compile('出血,视盘出血;出血,视盘出血').findall(str_n))>0:
#                     print(str_n)
                    df.at[i,column_name] = re.compile('出血,视盘出血;出血,视盘出血').sub('出血,视盘出血',str_n)
#                     print('__'+df_ym.at[i,column_name])
                elif len(re.compile('出血,玻璃体出血;出血,玻璃体出血').findall(str_n))>0:
#                     print(str_n)
                    df.at[i,column_name] = re.compile('出血,玻璃体出血;出血,玻璃体出血').sub('出血,玻璃体出血',str_n)
#                     print('__'+df_ym.at[i,column_name])
                elif len(re.compile('出血,眼底出血;出血,眼底出血').findall(str_n))>0:
#                     print(str_n)
                    df.at[i,column_name] = re.compile('出血,眼底出血;出血,眼底出血').sub('出血,眼底出血',str_n)
#                     print('__'+df_ym.at[i,column_name])                  
                else:
                    pass

In [ ]:
delete_func(df_ym,'left_出血location')

In [ ]:
delete_func(df_ym,'right_出血location')

In [ ]:
df_ym['left_出血location'].value_counts()  # C/D=0.5--

In [ ]:
df_ym['right_出血location'].value_counts()

In [ ]:
df_ym_jiao = df_ym[['影像描述','影描_gai','left_出血location','right_出血location']].drop_duplicates(keep='first')  # 去重
## 患者维度
print("'影描_gai'字段去重后——数据容量:{}".format(df_ym_jiao.shape[0]) )  # '影描_gai'字段去重后的数据容量
df_ym_jiao[:5]

In [ ]:
## 患者维度——含白内障
### '影描_gai'字段——含“白内障”标签的数据容量——去重后，唯一标签
df_ym_jiao_notduplicate = df_ym_jiao[df_ym_jiao['影描_gai'].str.contains('出血')]   # '影描_gai'字段去重后——含白内障的数据容量
print("'影描_gai'字段去重后_含出血location——数据容量:{}".format(df_ym_jiao_notduplicate.shape[0]))

In [ ]:
df_last_01 = df_ym_jiao.sample(n=5000,replace=False,random_state=None,axis=0)

In [ ]:
df_last_01[df_last_01['影描_gai'].str.contains('出血')].shape[0]

In [ ]:
output_path = r'/Users/edz/Documents/yandi_data/导出数据/影像描述'
output_file = r'/出血location_' + str(b.shape[0]) +'_20200409.xlsx'
writer = pd.ExcelWriter(output_path + output_file)
df_last_01.to_excel(writer,sheet_name='含出血location_5000',index=None)
df_ym_jiao_notduplicate.to_excel(writer,sheet_name='含出血location_去重_'+ str(df_ym_jiao_notduplicate.shape[0]),index=None)
df_ym_jiao.to_excel(writer,sheet_name='影描_gai_去重_'+str(df_ym_jiao.shape[0]),index=None)
writer.save()

In [ ]:
# 输出全量250625的糖网标签
df_ym_quan = df_ym[['来自检查id','影描_gai','left_出血location','right_出血location']]

In [ ]:
df_ym_quan.shape[0]

In [ ]:
# 导出新增疾病标签
output_path = r'/Users/edz/Documents/yandi_data/AI_data/zn_ai_影像描述标签'
output_file = r'/出血location_quan_250625.xlsx'
writer = pd.ExcelWriter(output_path + output_file)
df_ym_quan.to_excel(writer,sheet_name='出血location'+str(df_ym_quan.shape[0]),index=None)
writer.save()